In [ ]:
import os
pwd = os.getcwd()

pwd

In [ ]:
import pandas as pd
import numpy as np
import sys

if "/home/" in pwd:
    sys.path.insert(0, "./utils")
    sys.path.insert(0, "/mnt/d/cloud/unizg/OneDrive - Faculty of Graphic Arts of University of Zagreb/poligon/Pocket ML/utils")
elif "\\cloud\\" in pwd:
    sys.path.insert(0, "../utils")


from workbench import Stats

### Učitavanje podataka iz triju baza

Sve tri baze su unaprijed pripremljene preko `wikiart-setup.py`, `wikiart-emotions-setup.py` i moćnih ručno izrađenih alata za baratanje ovim bazama smještenih u `../../utils` direktoriju.

WikiArt baza je ta koja je pripremljena s već kategoriziranim stilovima. Ta kategorizacija nije nužno idealna i vjerojatno će ju trebati ponoviti, ali na probranom uzorku izložaka jer, čini mi se, nema razloga teretiti cijelu bazu već samo uzorak za trening i testiranje.


In [ ]:
wikiart = Stats(data_url="./data/wikiart_paintings.parquet", title="Wikiart")
emotions = Stats(data_url="./data/Wikiart-Emotions-All-t.parquet", title="Wikiart-emotions")
artemis = Stats(data_url="./data/artemis_dataset_release_v0-t2.parquet", title="ArtEmis")

### Spajanje WikiArt baze s Artemis bazom


In [ ]:
mappings = wikiart.raw_data.merge(artemis.raw_data.rename(columns={"emotion":"artemis", "Artist": "artistName", "Title": "title"})[["artistName", "title", "artemis"]], on=["artistName", "title"], how="left")
mappings.dropna(subset=["artemis"]).shape, artemis.raw_data.shape

### Spajanje WikiArt-Artemis baze s WikiArt_emotions bazom


In [ ]:
emotions_cols = emotions.raw_data[['emotion', 'Is painting', 'Face/body', 'Rating', "Title", "Artist", "Year", 'webUrl']].rename(columns={"emotion": "emotions", "Title": "title", "Artist": "artistName", "webUrl": "webUrl2"})
mappings_em = mappings.merge(emotions_cols, on=["artistName", "title"], how="left")

mappings_em.dropna(subset=["emotions"]).shape, emotions_cols.shape

### Čišćenje i optimiziranje


In [ ]:
# from styles import style_categories as styles
int_columns = ['Icon', 'Realism', 'Romanticism', 'Impressionism', 'Expressionism', 'Surrealism', 'Naive Art', 'Fauvism', 'Brutalism', 'Dada', 'Cubism',
    'Abstract Order', 'Abstract Chaos', 'Applied Art', 'Oriental Illustration', 'Photography', 'abstract', 'painting', 'drawing', 'composit', 'applied art', 'photography']
large_int_columns = [ 'contentId', 'artistContentId' ]
mappings_em[ int_columns ] = mappings_em[ int_columns ].astype(np.int8)
mappings_em[ large_int_columns ] = mappings_em[ large_int_columns ].astype(np.int64)

In [ ]:
mappings_em.year = mappings_em.year.apply(pd.to_numeric).astype("Int64")
mappings_em.sample(5)

In [ ]:
mappings_em = mappings_em.set_index("contentId")

In [ ]:
cat_columns = [ "style", "genre", "Is painting", "Face/body"]
mappings_em[ cat_columns ] = mappings_em[ cat_columns ].astype("category")
mappings_em.sample(5)

### Pohranjivanje kompozitne baze


In [ ]:
mappings_em.shape

In [ ]:
mappings_em[['artistName', 'title', 'year', 'style', 'genre', 'photography', 'artemis', 'emotions',
       'Is painting', 'Face/body', 'Rating', 'artistContentId',
       'artistUrl', 'url', 'localUrl', 'webUrl', 'webUrl2']].rename(columns={"url": "paintingUrl"}).to_parquet("./data/wikiart_composed.parquet")